<a href="https://colab.research.google.com/github/rauhalai/freeCodeCamp/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-07-12 12:27:57--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  76.6MB/s    in 0.3s    

2023-07-12 12:27:58 (76.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
## Data cleaning ##

# Users count of ratings
user_rating_count = df_ratings['user'].value_counts()
isbn_rating_count = df_ratings['isbn'].value_counts()

# Remove users with < 200 given ratings
df_ratings = df_ratings[~df_ratings['user'].isin(user_rating_count[user_rating_count < 200].index)]

# Remove books with < 100 ratings
df_ratings = df_ratings[~df_ratings['isbn'].isin(isbn_rating_count[isbn_rating_count < 100].index)]


In [5]:
# Pivot and matrix ratings data for model

df_ratings_pivot = df_ratings.pivot(index = 'isbn', columns = 'user', values = 'rating').fillna(0)
df_ratings_matrix = csr_matrix(df_ratings_pivot.values)


In [6]:
# Build KNN model

model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model.fit(df_ratings_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [7]:
# get book title from indice
def get_book_title(indice):

  # get book isbn
  pivot_book_isbn = df_ratings_pivot.index[indice]

  # get book title
  book_title = df_books.loc[df_books['isbn'] == pivot_book_isbn, 'title'].iloc[0]

  return book_title

In [8]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  # get book isbn
  book_isbn = df_books.loc[df_books['title'] == book, 'isbn'].iloc[0]

  # get pivot index
  book_pivot_index = df_ratings_pivot.index.get_loc(book_isbn)

  # get indices and distances of 6 nearest neighbours
  distance, indice = model.kneighbors(df_ratings_pivot.iloc[book_pivot_index, :].array.reshape(1, -1), n_neighbors=6)

  # get book titles from indices
  titles = []
  for book_indice in indice[0]:
    titles.append(get_book_title(book_indice))

  # format distance
  pd.options.display.float_format = '{:.15f}'.format

  # put distances and titles from indices to df
  df_recommends = pd.DataFrame({'r_indice':indice[0],'r_title': titles, 'r_distance':distance[0]})

  # sort by distance
  df_recommends = df_recommends.sort_values(by='r_distance', ascending=False)

  # drop 6th
  df_recommends = df_recommends[:-1]

  ### form returnable ###

  recommended_books = []

  # original book
  recommended_books.append(book)

  recommendations = []

  # insert recommendations and distances
  for index, row in df_recommends.iterrows():
    recommended_book_distance = [row['r_title'], row['r_distance']]
    recommendations.append(recommended_book_distance)

  recommended_books.append(recommendations)

  return recommended_books


In [9]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
